In [1]:
from bs4 import BeautifulSoup #Python Web scraping library
import requests # library to handle requests
import json # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.35 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.18 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   

Collecting Toronto Neighborhoods and their Postal codes by web scraping

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(url).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')

In [4]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighborhood  = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighborhood_var  = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighborhood_var = str(td_cell.text).strip()
            tag_a_Neighborhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighborhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighborhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighborhood.append(Neighborhood_var)

In [5]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighborhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighborhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighborhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighborhood_u.append(n_var)

In [9]:
toronto_dict = {'Borough':Borough_u,'Postcode':Postcode_u,'Neighborhood':Neighborhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
#df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Borough,Neighborhood,Postcode
0,North York,Victoria Village,M4A
1,North York,"Silver Hills, York Mills",M2L
2,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P
3,Downtown Toronto,Berczy Park,M5E
4,Downtown Toronto,Ryerson,M5B


In [15]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_coords=df_data_1

In [18]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_toronto = pd.merge(df_toronto, df_coords.rename(columns={'Postal Code':'Postcode'}), on='Postcode',  how='left')

In [21]:
df_toronto.head(10)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Victoria Village,M4A,43.725882,-79.315572
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
2,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
3,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
4,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
5,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
6,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,43.739416,-79.588437
7,Downtown Toronto,"Toronto Islands, Union Station",M5J,43.640816,-79.381752
8,North York,Bayview Village,M2K,43.786947,-79.385975
9,Etobicoke,"Humber Bay, King's Mill Park, Mimico NE, Old M...",M8Y,43.636258,-79.498509


Create a map of North York and its neighborhoods

In [22]:
address = 'North York,Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of North York ,Toronto are {}, {}.'.format(latitude_x, longitude_y))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of North York ,Toronto are 43.7709163, -79.4124102.


In [23]:
borough_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
borough_data.head(7)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Victoria Village,M4A,43.725882,-79.315572
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
2,North York,Bayview Village,M2K,43.786947,-79.385975
3,North York,Bedford Park,M5M,43.733283,-79.419750
4,North York,Downsview West,M3L,43.739015,-79.506944
5,North York,"Bathurst Manor, Wilson Heights",M3H,43.754328,-79.442259
6,North York,Humber Summit,M9L,43.756303,-79.565963


In [24]:
# create map of North York,Toronto using latitude and longitude values
map_scarb = folium.Map(location=[latitude_x, longitude_y], zoom_start=12)

In [25]:
# add markers to map
#for lat, lng, nei in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
for lat, lng, nei in zip(borough_data['Latitude'], borough_data['Longitude'], borough_data['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarb)  
    
map_scarb